# WEB SCRAPING 

Since our project relies on a working database of recipes we need to find the recipes and be able to input them into our database. An efficient way of doing this is through web scraping (while giving credit of course) online recipe sites.

This means that a large part of the project actually having data relies on this webscraper and the quality of recipes and information it can extract.

In [102]:
import selenium
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [12]:
URL = 'https://www.allrecipes.com/recipes/659/meat-and-poultry/chicken/chicken-breasts/'
r = requests.get(URL)

soup = BeautifulSoup(r.content, 'html5lib')

In [114]:
recipes = []
i = 0

#table = soup.find('div', attrs = {'id':'tax-sc__recirc-list_1-0-1'})

for row in soup.findAll('a',
                        attrs = {'class': ['comp mntl-card-list-items mntl-document-card mntl-card card card--no-image']}):
    try:
        recipe = {}
        nurl = row.get('href')
        nr = requests.get(nurl)
        nsoup = BeautifulSoup(nr.content, 'html5lib')

        name = nsoup.find('h1', attrs = {'id' : 'article-heading_1-0'}).text
        recipe['name'] = name[1:]

        details = nsoup.find('div', attrs = {'class':'mntl-recipe-details__content'})
        for nrow in details.findAll('div', attrs = {'class':'mntl-recipe-details__item'}):

            label = nrow.find('div', attrs = {'class' : 'mntl-recipe-details__label'}).text
            value = nrow.find('div', attrs = {'class' : 'mntl-recipe-details__value'}).text
            recipe[label[:-1]] = value

        ingredientsData = nsoup.find('ul', attrs = {'class' : 'mntl-structured-ingredients__list'})
        ingredients = []
        for nrow in ingredientsData.findAll('li', attrs = {'class' : 'mntl-structured-ingredients__list-item'}):

            ingredient = {}
            quantity = nrow.find('span', attrs = {'data-ingredient-quantity' : 'true'}).text
            unit = nrow.find('span', attrs = {'data-ingredient-unit' : 'true'}).text
            ing_name = nrow.find('span', attrs = {'data-ingredient-name' : 'true'}).text

            ingredient['quantity'] = quantity
            ingredient['unit'] = unit
            ingredient['name'] = ing_name

            ingredients.append(ingredient)
            recipe['ingredients'] = ingredients

        directions = nsoup.find('div', attrs = {'id' : 'recipe__steps-content_1-0'}).text
        directions = [l for l in directions.splitlines() if l]
        dirdict = {}
        for i in range(len(directions)): dirdict[f'{i}'] = directions[i];

        recipe['directions'] = directions

        recipe['url'] = nurl

        recipes.append(recipe)
    except:
        continue

In [115]:
df = pd.DataFrame.from_dict(recipes)
df.to_csv('recipes.csv')